In [3]:
import pandas as pd
#import pymysql
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta

load_dotenv()


True

In [3]:
# data = pd.read_csv("test.csv", header=None)
# data.head()

In [4]:
host = os.getenv("HOST")
db_username = os.getenv("DB_USERNAME")
password = os.getenv("PASSWORD")
db_name = os.getenv("LOCAL_DB")
port = os.getenv("PORT")

engine = create_engine(f'mysql+pymysql://root:{password}@{host}/{db_name}')

In [5]:
past_days=37
sql_query = "SELECT * FROM scada_data WHERE DATE(datetime) = %(serach_date)s"
my_params={"serach_date":datetime.strftime(datetime.now() - timedelta(past_days), '%Y-%m-%d')}
filtered_data = pd.read_sql(sql_query,con=engine, params=my_params)

filtered_data.head()

,index,datetime,event
0,7051,2022-11-13 00:02:38,TORO 110V DC Normal
1,7052,2022-11-13 00:02:38,TORO 48V DC Normal
2,7053,2022-11-13 00:02:38,TORO UMEME CHARGER Normal
3,7054,2022-11-13 00:26:10,KWAD 33 REAC CB Closed N
4,7055,2022-11-13 00:26:33,MASK 33 REAC 1 CB Closed N


In [6]:
def clean_data(data):
    # rename columns
    data = data.drop("index", axis=1)
    data.columns = ['datetime', 'event']
    # filter data 
    data = data[(data.datetime != "time") | (data.event != "event") ]
    #data["event"] = data["event"].str.lower().str.replace(" ","_")
    data["event"] = data["event"].str.lower()

    # covert datetime column to datetime object. SHOULD BE DONE AFTER FILTERING. ELSE, ERRORS
    data.datetime = pd.to_datetime(data.datetime)
    data["date"] = data.datetime.dt.date
    data["month"] = data.datetime.dt.month_name()
    data['weekday'] = data.datetime.dt.day_name()
    data["time"] = data.datetime.dt.time   

    return data



In [7]:
classify_list = ["trip cct fault","station supply fail","dc power failure","cb lockout locked","cb backup trip",
"cb back prot trip","dc fault","diff comm fail fault", "cb err status","sf6 lockout", "relay fault"]
def classify_events(row):
    #print(row)  
    for s in classify_list:
        if s in str(row):
            return "critical"

    # mind the indentation. Else, the for loop might check only the first event in the "classify_list"
    else:
        return "normal"

In [8]:
data = clean_data(filtered_data)
data.head()

,datetime,event,date,month,weekday,time
0,2022-11-13 00:02:38,toro 110v dc normal,2022-11-13,November,Sunday,00:02:38
1,2022-11-13 00:02:38,toro 48v dc normal,2022-11-13,November,Sunday,00:02:38
2,2022-11-13 00:02:38,toro umeme charger normal,2022-11-13,November,Sunday,00:02:38
3,2022-11-13 00:26:10,kwad 33 reac cb closed n,2022-11-13,November,Sunday,00:26:10
4,2022-11-13 00:26:33,mask 33 reac 1 cb closed n,2022-11-13,November,Sunday,00:26:33


In [9]:

data['type'] = data['event'].apply(lambda row: classify_events(row))
data.head()

,datetime,event,date,month,weekday,time,type
0,2022-11-13 00:02:38,toro 110v dc normal,2022-11-13,November,Sunday,00:02:38,normal
1,2022-11-13 00:02:38,toro 48v dc normal,2022-11-13,November,Sunday,00:02:38,normal
2,2022-11-13 00:02:38,toro umeme charger normal,2022-11-13,November,Sunday,00:02:38,normal
3,2022-11-13 00:26:10,kwad 33 reac cb closed n,2022-11-13,November,Sunday,00:26:10,normal
4,2022-11-13 00:26:33,mask 33 reac 1 cb closed n,2022-11-13,November,Sunday,00:26:33,normal


In [10]:
#critical_data.event.value_counts()

In [11]:
data["substation_acronym"] = data["event"].apply(lambda x: x.split()[0])
data["fault_voltage"] = data["event"].apply(lambda x: x.split()[1])
data.head()

,datetime,event,date,month,weekday,time,type,substation_acronym,fault_voltage
0,2022-11-13 00:02:38,toro 110v dc normal,2022-11-13,November,Sunday,00:02:38,normal,toro,110v
1,2022-11-13 00:02:38,toro 48v dc normal,2022-11-13,November,Sunday,00:02:38,normal,toro,48v
2,2022-11-13 00:02:38,toro umeme charger normal,2022-11-13,November,Sunday,00:02:38,normal,toro,umeme
3,2022-11-13 00:26:10,kwad 33 reac cb closed n,2022-11-13,November,Sunday,00:26:10,normal,kwad,33
4,2022-11-13 00:26:33,mask 33 reac 1 cb closed n,2022-11-13,November,Sunday,00:26:33,normal,mask,33


In [12]:
critical_data = data[data.type == 'critical']
print(critical_data.shape)
critical_data.head()

(157, 9)


,datetime,event,date,month,weekday,time,type,substation_acronym,fault_voltage
17,2022-11-13 00:48:42,isba 11 gen2 cb err status 00,2022-11-13,November,Sunday,00:48:42,critical,isba,11
38,2022-11-13 01:10:22,nmve 132 nmvs cb trip cct fault n,2022-11-13,November,Sunday,01:10:22,critical,nmve,132
39,2022-11-13 01:10:23,nmve 132 nmvs cb trip cct fault n,2022-11-13,November,Sunday,01:10:23,critical,nmve,132
48,2022-11-13 02:08:49,toro 33 bgri cb lockout locked,2022-11-13,November,Sunday,02:08:49,critical,toro,33
70,2022-11-13 03:11:11,mbrn 33 kble cb aux dc fault,2022-11-13,November,Sunday,03:11:11,critical,mbrn,33


In [13]:
# substations = data.drop_duplicates(subset=['substation_acronym'], keep='first')
# substations['substation_acronym'].to_csv("substation_acronyms2.csv")

In [14]:
# excel_data = pd.read_excel("substation details.xlsx",sheet_name="substations", index_col='no')
# excel_data.reset_index(inplace=True)
# excel_data.head()

In [15]:
#data.to_csv("processed_data.csv")

## Combine dataframe. include substation details

In [16]:
## read final_data
# substation_data_columns = ['substation_acronym',"critical level","substation name"]
# substation_data = pd.read_excel("substation details.xlsx", index_col="no",sheet_name=0)
# final_data = data.merge(substation_data[substation_data_columns], how="inner", on="substation_acronym")
# final_data.head()

In [17]:
#final_data.to_csv("final_data.csv")

In [18]:
sql_qyery = "SELECT * FROM substation_data"
substation_data = pd.read_sql(sql_qyery,con=engine)
substation_data.drop(['no','voltage','comments'], axis=1, inplace=True)
substation_data.head()

,substation_acronym,substation_name,voltage_levels,installed_capacity_(mw),location,generators,critical_level
0,mskw,masaka west,3,540.0,central,0,3
1,nkda,nkenda,3,40.0,western,1,4
2,owfs,owen falls,3,380.0,eastern,1,5
3,homa,hoima,2,40.0,western,1,3
4,ofex,owen falls extension,2,NaN,eastern,1,5


In [19]:
final_data = data.merge(substation_data, how="inner", on="substation_acronym")
final_data.head()

,datetime,event,date,month,weekday,time,type,substation_acronym,fault_voltage,substation_name,voltage_levels,installed_capacity_(mw),location,generators,critical_level
0,2022-11-13 00:02:38,toro 110v dc normal,2022-11-13,November,Sunday,00:02:38,normal,toro,110v,tororo,2,160.0,eastern,1,5
1,2022-11-13 00:02:38,toro 48v dc normal,2022-11-13,November,Sunday,00:02:38,normal,toro,48v,tororo,2,160.0,eastern,1,5
2,2022-11-13 00:02:38,toro umeme charger normal,2022-11-13,November,Sunday,00:02:38,normal,toro,umeme,tororo,2,160.0,eastern,1,5
3,2022-11-13 02:08:49,toro 33 bgri cb main prot trip,2022-11-13,November,Sunday,02:08:49,normal,toro,33,tororo,2,160.0,eastern,1,5
4,2022-11-13 02:08:49,toro 33 bgri cb lockout locked,2022-11-13,November,Sunday,02:08:49,critical,toro,33,tororo,2,160.0,eastern,1,5


In [20]:
final_data["Rank"] = final_data[["critical_level","generators","voltage_levels","installed_capacity_(mw)"]]\
                        .apply(tuple,axis=1).rank(method='dense',ascending=False).astype(int)
final_data.sort_values("Rank")
final_data.head()

,datetime,event,date,month,weekday,time,type,substation_acronym,fault_voltage,substation_name,voltage_levels,installed_capacity_(mw),location,generators,critical_level,Rank
0,2022-11-13 00:02:38,toro 110v dc normal,2022-11-13,November,Sunday,00:02:38,normal,toro,110v,tororo,2,160.0,eastern,1,5,8
1,2022-11-13 00:02:38,toro 48v dc normal,2022-11-13,November,Sunday,00:02:38,normal,toro,48v,tororo,2,160.0,eastern,1,5,8
2,2022-11-13 00:02:38,toro umeme charger normal,2022-11-13,November,Sunday,00:02:38,normal,toro,umeme,tororo,2,160.0,eastern,1,5,8
3,2022-11-13 02:08:49,toro 33 bgri cb main prot trip,2022-11-13,November,Sunday,02:08:49,normal,toro,33,tororo,2,160.0,eastern,1,5,8
4,2022-11-13 02:08:49,toro 33 bgri cb lockout locked,2022-11-13,November,Sunday,02:08:49,critical,toro,33,tororo,2,160.0,eastern,1,5,8


In [23]:
final_data[final_data["Rank"] == 3].head()

,datetime,event,date,month,weekday,time,type,substation_acronym,fault_voltage,substation_name,voltage_levels,installed_capacity_(mw),location,generators,critical_level,Rank
575,2022-11-13 11:11:07,bjgl station control authority remote,2022-11-13,November,Sunday,11:11:07,normal,bjgl,station,bujagali,2,500.0,eastern,1,5,3
576,2022-11-13 11:12:28,bjgl 132 nble 2 cb open,2022-11-13,November,Sunday,11:12:28,normal,bjgl,132,bujagali,2,500.0,eastern,1,5,3
577,2022-11-13 11:13:06,bjgl 132 nble 2 cb trip cct1 fault,2022-11-13,November,Sunday,11:13:06,normal,bjgl,132,bujagali,2,500.0,eastern,1,5,3
578,2022-11-13 11:13:06,bjgl 132 nble 2 cb trip cct2 fault,2022-11-13,November,Sunday,11:13:06,normal,bjgl,132,bujagali,2,500.0,eastern,1,5,3
579,2022-11-13 17:14:45,bjgl 132 nble 2 cb closed,2022-11-13,November,Sunday,17:14:45,normal,bjgl,132,bujagali,2,500.0,eastern,1,5,3
